<a href="https://colab.research.google.com/github/Anirudh2465/Semester1/blob/ECG-Analysis/Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy scipy wfdb vmdpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wfdb
from scipy import stats
from vmdpy import VMD
from sklearn import svm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from scipy.stats import skew, kurtosis
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

In [ ]:
# Define the VMD parameters
alpha = 2000  # Bandwidth constraint
tau = 0.  # Noise-tolerance (no strict fidelity enforcement)
K = 3  # 3 modes
DC = 0  # No DC part imposed
init = 1  # Initialize omegas uniformly
tol = 1e-7  # Tolerance

L_data=[109,101,233,104,207,220]

# Initialize a list to store the results
u_chunks = []
ecg_data=[]
for j in L_data:
  # Load the MIT-BIH Arrhythmia dataset
  record = wfdb.rdrecord(f'/content/{j}')

  # Get the ECG data
  ecg_data1 = record.p_signal.flatten()

  ecg_data.append(ecg_data1)

  # Define the size of each chunk
  chunk_size = 10000  # Adjust this value based on your resources

  # Calculate the number of chunks
  num_chunks = len(ecg_data1) // chunk_size



  # Loop over each chunk
  for i in range(num_chunks):
      # Get the start and end indices of the chunk
      start = i * chunk_size
      end = start + chunk_size

      # Get the chunk of data
      chunk = ecg_data1[start:end]

      # Apply VMD to the chunk
      u_chunk, u_hat_chunk, omega_chunk = VMD(chunk, alpha, tau, K, DC, init, tol)

      # Append the result to the list
      u_chunks.append(u_chunk)


      print("Done VDM",j, "chunk",i+1)

# Concatenate the chunks along the second axis
u = np.concatenate(u_chunks, axis=1)
ecg_data=np.concatenate(ecg_data)

def normalize(data):
    mean = np.mean(data)
    std = np.std(data)
    return (data - mean) / std

def signal_to_noise_ratio(signal, noise):
    signal_power = np.mean(np.square(signal))
    noise_power = np.mean(np.square(noise))
    return 10 * np.log10(signal_power / noise_power)

u=normalize(u)

# Plot the original ECG data and the IMFs
plt.figure(figsize=(12, 8))
plt.subplot(len(u) + 1, 1, 1)
plt.plot(ecg_data)
plt.title('Original ECG data')

for i, IMF in enumerate(u, start=2):
    plt.subplot(len(u) + 1, 1, i)
    plt.plot(IMF)
    plt.title(f'IMF {i - 1}')

plt.tight_layout()
plt.show()

snrval = signal_to_noise_ratio(u, ecg_data - u)
print(f'Signal-to-Noise Ratio after VMD: {snrval}')

# Initialize an empty list to store the features
features = []

# Initialize an empty list to store the labels
labels = []

for j in L_data:
  # Read the annotation
  annotation = wfdb.rdann(f'/content/{j}', 'atr')

  # Loop over each beat location
  for i in range(1, len(annotation.sample)):
    if annotation.symbol[i] in ['N','L','R','j','e','A','S','J','a','!','V','E','[',']','F','f','/','Q']:
        # Get the start and end of the beat
        start = annotation.sample[i-1]
        end = annotation.sample[i]

        # Initialize a list to store the features for this beat
        beat_features = []

        # Loop over each IMF
        for imf in u:
            # Segment the IMF at the beat location
            segment = imf[start:end]

            # Calculate the mean, standard deviation, skewness, and kurtosis of the segment
            mean = np.mean(segment)
            std = np.std(segment)
            skewness = skew(segment)
            kurt = kurtosis(segment)

            # Append the features to the beat_features list
            beat_features.extend([mean, std, skewness, kurt])

        # Append the beat_features to the features list
        features.append(beat_features)

        # Append the label to the labels list
        labels.append(annotation.symbol[i])
  print("Feature extraction done for data set : ", j)
# Convert the lists to NumPy arrays
features = np.array(features)
labels = np.array(labels)

1. **VMD Parameters**: You're defining parameters for Variational Mode Decomposition (VMD), a method for decomposing a signal into multiple components, or modes. The parameters include the bandwidth constraint (`alpha`), noise tolerance (`tau`), number of modes (`K`), DC part (`DC`), initialization method (`init`), and tolerance (`tol`).

2. **Data Loading**: You're loading ECG data from the MIT-BIH Arrhythmia dataset. The dataset is divided into chunks of size 10000, and VMD is applied to each chunk. The results are stored in `u_chunks`.

3. **Signal Normalization**: After applying VMD to all chunks and concatenating the results, you normalize the signal. Normalization is a common preprocessing step in machine learning that makes the data have zero mean and unit variance.

4. **Signal-to-Noise Ratio (SNR)**: You're calculating the SNR of the normalized signal. SNR is a measure of signal strength relative to background noise. The higher the ratio, the less obtrusive the background noise is.

5. **Feature Extraction**: For each beat in the ECG data, you're extracting features from the output of VMD. These features include the mean, standard deviation, skewness, and kurtosis of each Intrinsic Mode Function (IMF) segment corresponding to the beat. The features are stored in `features`, and the corresponding labels are stored in `labels`.

6. **Data Conversion**: Finally, you're converting the lists of features and labels to NumPy arrays for further processing. This is a common practice as many machine learning algorithms require input data to be in the form of NumPy arrays.

This is a comprehensive approach to signal processing and feature extraction for ECG data.

In [ ]:
# Scale the features
scaler = StandardScaler()
val = scaler.fit_transform(features)

smote = SMOTE(k_neighbors=1)

# Fit the SMOTE transformer to the data
smote.fit(val, labels)

# Resample the data
val_resampled, labels_resampled = smote.fit_resample(val, labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(val_resampled, labels_resampled, test_size=0.2, random_state=42)

In [ ]:
model1 = HistGradientBoostingClassifier()
model1.fit(X_train, y_train)
print("Model 1 trained")

pred1 = model1.predict(X_test)
accuracy = np.mean(pred1 == y_test)
print(f'Accuracy of model 1: {accuracy}')


# Calculate precision, recall, and F1 score
precision = precision_score(y_test, pred1, average='weighted')
recall = recall_score(y_test, pred1, average='weighted')
f1 = f1_score(y_test, pred1, average='weighted')

# Print the scores
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Generate confusion matrix
matrix = confusion_matrix(y_test, pred1)

# Create a heatmap
sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=L2,
            yticklabels=L)

plt.title('Confusion Matrix')
plt.show()

In [ ]:
model2 = svm.SVC()
model2.fit(X_train,y_train)
print("Model 2 trained")

pred2 = model2.predict(X_test)
accuracy = np.mean(pred2 == y_test)
print(f'Accuracy of model 2: {accuracy}')


# Calculate precision, recall, and F1 score
precision = precision_score(y_test, pred2, average='weighted')
recall = recall_score(y_test, pred2, average='weighted')
f1 = f1_score(y_test, pred2, average='weighted')

# Print the scores
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Generate confusion matrix
matrix = confusion_matrix(y_test, pred2)

# Create a heatmap
sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=L2,
            yticklabels=L)

plt.title('Confusion Matrix')
plt.show()

In [ ]:
model3 = RandomForestClassifier()
model3.fit(X_train,y_train)
print("Model 3 trained")

pred3 = model3.predict(X_test)
accuracy = np.mean(pred3 == y_test)
print(f'Accuracy of model 3: {accuracy}')


# Calculate precision, recall, and F1 score
precision = precision_score(y_test, pred3, average='weighted')
recall = recall_score(y_test, pred3, average='weighted')
f1 = f1_score(y_test, pred3, average='weighted')

# Print the scores
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Generate confusion matrix
matrix = confusion_matrix(y_test, pred3)

# Create a heatmap
sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=L2,
            yticklabels=L)

plt.title('Confusion Matrix')
plt.show()

In [ ]:
model4 = DecisionTreeClassifier()
model4.fit(X_train, y_train)


pred4 = model4.predict(X_test)
accuracy = np.mean(pred4 == y_test)
print(f'Accuracy of Decision Tree model: {accuracy}')

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, pred4, average='weighted')
recall = recall_score(y_test, pred4, average='weighted')
f1 = f1_score(y_test, pred4, average='weighted')

# Print the scores
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Generate confusion matrix
matrix4 = confusion_matrix(y_test, pred4)

# Create a heatmap
sns.heatmap(matrix4, annot=True, fmt='d', cmap='Blues',
            xticklabels=L2,
            yticklabels=L)

plt.title('Confusion Matrix for Decision Tree')
plt.show()

1. `scaler = StandardScaler()`: This line initializes a `StandardScaler` object from the `sklearn.preprocessing` module. The `StandardScaler` standardizes features by removing the mean and scaling to unit variance.

2. `val = scaler.fit_transform(features)`: The `fit_transform` method computes the mean and standard deviation on `features` for later scaling (fit part) and then scales `features` (transform part). The transformed data is stored in `val`.

3. `smote = SMOTE(k_neighbors=1)`: This line initializes a `SMOTE` object from the `imblearn.over_sampling` module with `k_neighbors` set to 1. SMOTE stands for Synthetic Minority Over-sampling Technique. It is a technique used to handle class imbalance. It works by creating synthetic samples from the minor class instead of creating copies.

4. `smote.fit(val, labels)`: This line fits the SMOTE object to the data. It doesn't change the data but allows the object to gather necessary information about the data.

5. `val_resampled, labels_resampled = smote.fit_resample(val, labels)`: The `fit_resample` method resamples the dataset. It applies the synthetic over-sampling process to the dataset and returns the resampled feature set `val_resampled` and label set `labels_resampled`.

6. `X_train, X_test, y_train, y_test = train_test_split(val_resampled, labels_resampled, test_size=0.2, random_state=42)`: This line splits the resampled dataset into training and test sets. 80% of the data will be used for training and 20% for testing.

7. `model1 = HistGradientBoostingClassifier()`: This line initializes a `HistGradientBoostingClassifier` object from the `sklearn.ensemble` module. This is a histogram-based Gradient Boosting Classification Tree.

8. `model1.fit(X_train, y_train)`: This line fits the model to the training data.

9. `pred1 = model1.predict(X_test)`: This line uses the fitted model to make predictions on the test data.

10. `accuracy = np.mean(pred1 == y_test)`: This line calculates the accuracy of the model by comparing the predicted labels to the true labels.

11. `precision = precision_score(y_test, pred1, average='weighted')`: This line calculates the weighted precision of the model. Precision is the ratio of correctly predicted positive observations to the total predicted positive observations.

12. `recall = recall_score(y_test, pred1, average='weighted')`: This line calculates the weighted recall of the model. Recall (Sensitivity) - the ratio of correctly predicted positive observations to all observations in actual class.

13. `f1 = f1_score(y_test, pred1, average='weighted')`: This line calculates the weighted F1 score of the model. The F1 score is the harmonic mean of precision and recall.

14. `matrix = confusion_matrix(y_test, pred1)`: This line generates a confusion matrix, which is a table that is often used to describe the performance of a classification model on a set of test data for which the true values are known.

15. The last few lines of your code are used to create a heatmap of the confusion matrix using the seaborn library. This provides a visual representation of the performance of your model. The x and y labels of the heatmap represent the classes of your data. The color and the number in each cell represent the number of instances of actual class (row) predicted as the predicted class (column).
